# OpenSearch Korean Neural Sparse Model - Inference Test

학습된 한국어 Neural Sparse 모델을 로컬에서 테스트합니다.

## 테스트 항목
1. 모델 로드
2. 문서 인코딩 (BERT 기반)
3. 쿼리 인코딩 (IDF lookup - Inference-Free)
4. 유사도 계산 및 검색
5. 성능 벤치마크

## 1. 환경 설정 및 라이브러리 로드

In [ ]:
# Add parent directory to path for src imports
import sys
from pathlib import Path

# Add project root to path
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")


In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
from torch import nn

# Transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForMaskedLM

# GPU/CPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"디바이스: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 2. 모델 정의

In [ ]:
class OpenSearchDocEncoder(nn.Module):
    """
    OpenSearch Neural Sparse Document Encoder (Doc-only mode)
    
    문서를 sparse vector로 인코딩하는 모델입니다.
    """
    def __init__(self, model_name="klue/bert-base"):
        super().__init__()
        
        # BERT 기반 인코더
        self.config = AutoConfig.from_pretrained(model_name)
        self.bert = AutoModelForMaskedLM.from_pretrained(model_name)
        
        self.vocab_size = self.config.vocab_size
        
        # Log saturation activation: log(1 + ReLU(x))
        self.activation = lambda x: torch.log1p(torch.relu(x))
    
    def forward(self, input_ids, attention_mask, return_dict=False):
        """
        Forward pass
        
        Args:
            input_ids: (batch_size, seq_len)
            attention_mask: (batch_size, seq_len)
        
        Returns:
            sparse_vector: (batch_size, vocab_size)
        """
        # BERT MLM head output
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        
        # Logits: (batch_size, seq_len, vocab_size)
        logits = outputs.logits
        
        # Apply activation: log(1 + ReLU(logits))
        activated = self.activation(logits)
        
        # Max pooling over sequence length
        # (batch_size, seq_len, vocab_size) → (batch_size, vocab_size)
        sparse_vector = torch.max(
            activated * attention_mask.unsqueeze(-1),
            dim=1
        ).values
        
        if return_dict:
            return {'output': sparse_vector}
        
        return sparse_vector

print("✓ 모델 클래스 정의 완료")

## 3. 저장된 모델 로드

In [ ]:
# 모델 디렉토리 경로
MODEL_DIR = "./models/opensearch-korean-neural-sparse-v1"

# 파일 존재 확인
required_files = [
    "pytorch_model.bin",
    "idf.json",
    "config.json",
    "tokenizer.json",
    "vocab.txt"
]

print("📦 모델 파일 확인...")
for filename in required_files:
    filepath = os.path.join(MODEL_DIR, filename)
    if os.path.exists(filepath):
        size = os.path.getsize(filepath) / 1024
        print(f"  ✓ {filename:25s} ({size:>10.2f} KB)")
    else:
        print(f"  ✗ {filename:25s} [NOT FOUND]")
        raise FileNotFoundError(f"Required file not found: {filename}")

print("\n" + "="*60)
print("모델 로딩 중...")
print("="*60)

# 1. Config 로드
with open(os.path.join(MODEL_DIR, "config.json"), 'r', encoding='utf-8') as f:
    model_config = json.load(f)

print("\n✓ Config 로드 완료")
print(f"  모델 타입: {model_config['model_type']}")
print(f"  베이스 모델: {model_config['base_model']}")
print(f"  Vocab size: {model_config['vocab_size']:,}")
print(f"  Max length: {model_config['max_seq_length']}")

# 2. Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
print(f"\n✓ Tokenizer 로드 완료")
print(f"  Vocab size: {tokenizer.vocab_size:,}")

# 3. IDF 딕셔너리 로드
with open(os.path.join(MODEL_DIR, "idf.json"), 'r', encoding='utf-8') as f:
    idf_dict = json.load(f)

print(f"\n✓ IDF 딕셔너리 로드 완료")
print(f"  토큰 수: {len(idf_dict):,}")
print(f"  평균 IDF: {np.mean(list(idf_dict.values())):.4f}")

# 4. Document Encoder 모델 로드
base_model = model_config['base_model']
doc_encoder = OpenSearchDocEncoder(base_model)

# 학습된 가중치 로드
state_dict = torch.load(
    os.path.join(MODEL_DIR, "pytorch_model.bin"),
    map_location=device
)
doc_encoder.load_state_dict(state_dict)
doc_encoder = doc_encoder.to(device)
doc_encoder.eval()  # Evaluation mode

print(f"\n✓ Document Encoder 로드 완료")
print(f"  Parameters: {sum(p.numel() for p in doc_encoder.parameters()):,}")
print(f"  Device: {device}")

print("\n" + "="*60)
print("✅ 모델 로드 완료!")
print("="*60)

## 4. Inference 함수 정의

In [ ]:
def encode_document(text, model, tokenizer, device, max_length=128):
    """
    문서를 sparse vector로 인코딩 (모델 사용 - 느림)
    
    Args:
        text: 문서 텍스트
        model: Document encoder 모델
        tokenizer: Tokenizer
        device: 디바이스
        max_length: 최대 시퀀스 길이
    
    Returns:
        sparse_vector: (vocab_size,) numpy array
    """
    model.eval()
    
    # 토큰화
    encoded = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    
    # Inference
    with torch.no_grad():
        sparse_vec = model(input_ids, attention_mask)
    
    return sparse_vec.cpu().numpy()[0]


def encode_query_inference_free(text, tokenizer, idf_dict, max_length=128):
    """
    쿼리를 sparse vector로 인코딩 (IDF lookup - Inference-Free! 매우 빠름)
    
    Args:
        text: 쿼리 텍스트
        tokenizer: Tokenizer
        idf_dict: IDF 딕셔너리
        max_length: 최대 시퀀스 길이
    
    Returns:
        sparse_vector: (vocab_size,) numpy array
    """
    # 토큰화
    tokens = tokenizer.encode(
        text,
        add_special_tokens=False,
        max_length=max_length,
        truncation=True
    )
    
    # IDF lookup
    sparse_vec = np.zeros(tokenizer.vocab_size)
    for token_id in tokens:
        token_str = tokenizer.decode([token_id])
        if token_str in idf_dict:
            sparse_vec[token_id] = idf_dict[token_str]
    
    return sparse_vec


def get_top_tokens(sparse_vec, tokenizer, top_k=15):
    """
    Sparse vector에서 상위 토큰 추출
    
    Args:
        sparse_vec: (vocab_size,) numpy array
        tokenizer: Tokenizer
        top_k: 상위 k개
    
    Returns:
        List of (token, value) tuples
    """
    top_indices = np.argsort(sparse_vec)[-top_k:][::-1]
    top_values = sparse_vec[top_indices]
    
    top_tokens = []
    for idx, val in zip(top_indices, top_values):
        if val > 0:
            token = tokenizer.decode([idx])
            top_tokens.append((token, val))
    
    return top_tokens


def compute_similarity(query_vec, doc_vec):
    """
    쿼리와 문서의 유사도 계산 (Dot product)
    
    Args:
        query_vec: (vocab_size,) numpy array
        doc_vec: (vocab_size,) numpy array
    
    Returns:
        similarity score (float)
    """
    return np.dot(query_vec, doc_vec)


print("✓ Inference 함수 정의 완료")

## 5. 문서 인코딩 테스트

In [ ]:
# 테스트 문서
test_documents = [
    "OpenSearch는 강력한 오픈소스 검색 및 분석 엔진입니다. 벡터 검색과 neural sparse 검색을 지원합니다.",
    "한국어 자연어 처리는 형태소 분석, 품사 태깅, 개체명 인식 등을 포함합니다. KoNLPy와 같은 라이브러리가 널리 사용됩니다.",
    "딥러닝 모델 학습을 위해서는 충분한 데이터와 컴퓨팅 자원이 필요합니다. GPU를 사용하면 학습 속도가 크게 향상됩니다.",
    "ChatGPT는 OpenAI가 개발한 대화형 AI 모델입니다. GPT 아키텍처를 기반으로 하며 다양한 작업을 수행할 수 있습니다.",
    "벡터 검색은 임베딩 기반으로 의미적 유사도를 계산하여 검색합니다. Dense retrieval 방식이라고도 불립니다.",
]

print("="*60)
print("📄 문서 인코딩 테스트 (Model Inference)")
print("="*60)

doc_vectors = []
encoding_times = []

for i, doc in enumerate(test_documents, 1):
    print(f"\n[문서 {i}]")
    print(f"텍스트: {doc[:60]}...")
    
    # 인코딩 시간 측정
    start_time = time.time()
    sparse_vec = encode_document(doc, doc_encoder, tokenizer, device)
    elapsed = time.time() - start_time
    encoding_times.append(elapsed)
    
    doc_vectors.append(sparse_vec)
    
    # 통계
    non_zero = np.count_nonzero(sparse_vec)
    sparsity = (1 - non_zero / len(sparse_vec)) * 100
    l1_norm = np.sum(np.abs(sparse_vec))
    
    print(f"  희소성: {sparsity:.2f}% (non-zero: {non_zero:,}/{len(sparse_vec):,})")
    print(f"  L1 Norm: {l1_norm:.2f}")
    print(f"  인코딩 시간: {elapsed*1000:.2f}ms")
    
    # 상위 토큰
    print(f"  상위 토큰:")
    top_tokens = get_top_tokens(sparse_vec, tokenizer, top_k=10)
    for j, (token, value) in enumerate(top_tokens, 1):
        print(f"    {j:2d}. {token:15s} ({value:.4f})")

print(f"\n평균 인코딩 시간: {np.mean(encoding_times)*1000:.2f}ms")

## 6. 쿼리 인코딩 테스트 (Inference-Free)

In [ ]:
# 테스트 쿼리
test_queries = [
    "OpenSearch 벡터 검색",
    "한국어 자연어 처리 기술",
    "딥러닝 GPU 학습",
    "ChatGPT LLM 모델",
    "검색 엔진 최적화",
    "인공지능 프롬프트 엔지니어링",
]

print("="*60)
print("📝 쿼리 인코딩 테스트 (Inference-Free: Tokenizer + IDF Lookup)")
print("="*60)

query_vectors = []
query_encoding_times = []

for i, query in enumerate(test_queries, 1):
    print(f"\n[쿼리 {i}] {query}")
    
    # 인코딩 시간 측정
    start_time = time.time()
    sparse_vec = encode_query_inference_free(query, tokenizer, idf_dict)
    elapsed = time.time() - start_time
    query_encoding_times.append(elapsed)
    
    query_vectors.append(sparse_vec)
    
    # 통계
    non_zero = np.count_nonzero(sparse_vec)
    sparsity = (1 - non_zero / len(sparse_vec)) * 100
    
    print(f"  희소성: {sparsity:.2f}% (non-zero: {non_zero}/{len(sparse_vec):,})")
    print(f"  인코딩 시간: {elapsed*1000:.4f}ms ⚡ (매우 빠름!)")
    
    # 상위 토큰
    print(f"  상위 토큰:")
    top_tokens = get_top_tokens(sparse_vec, tokenizer, top_k=10)
    for j, (token, value) in enumerate(top_tokens, 1):
        print(f"    {j:2d}. {token:15s} ({value:.4f})")

print(f"\n평균 쿼리 인코딩 시간: {np.mean(query_encoding_times)*1000:.4f}ms")
print(f"\n💡 문서 인코딩 대비 속도: {np.mean(encoding_times)/np.mean(query_encoding_times):.1f}배 빠름!")

## 7. 검색 테스트 (유사도 계산)

In [ ]:
print("="*60)
print("🔍 검색 테스트 (Query-Document Similarity)")
print("="*60)

# 각 쿼리에 대해 가장 유사한 문서 찾기
for i, query in enumerate(test_queries):
    print(f"\n{'='*60}")
    print(f"쿼리: {query}")
    print(f"{'='*60}")
    
    query_vec = query_vectors[i]
    
    # 모든 문서와의 유사도 계산
    similarities = []
    for j, doc_vec in enumerate(doc_vectors):
        sim = compute_similarity(query_vec, doc_vec)
        similarities.append((j, sim))
    
    # 유사도 순으로 정렬
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # 상위 3개 문서 출력
    print("\n검색 결과 (상위 3개):")
    for rank, (doc_idx, score) in enumerate(similarities[:3], 1):
        print(f"\n  [{rank}위] 유사도: {score:.4f}")
        print(f"       문서: {test_documents[doc_idx][:80]}...")

## 8. 성능 벤치마크

In [ ]:
print("="*60)
print("⚡ 성능 벤치마크")
print("="*60)

# 대량 쿼리 처리 속도 테스트
num_queries = 100
test_query_text = "OpenSearch 벡터 검색 최적화"

print(f"\n테스트: {num_queries}개 쿼리 인코딩 (Inference-Free)")
start_time = time.time()
for _ in range(num_queries):
    _ = encode_query_inference_free(test_query_text, tokenizer, idf_dict)
elapsed = time.time() - start_time

print(f"총 시간: {elapsed:.4f}s")
print(f"평균 시간: {elapsed/num_queries*1000:.4f}ms per query")
print(f"처리량: {num_queries/elapsed:.2f} queries/sec")

# 문서 인코딩 속도 테스트
num_docs = 10
test_doc_text = "OpenSearch는 강력한 검색 엔진입니다."

print(f"\n테스트: {num_docs}개 문서 인코딩 (Model Inference)")
start_time = time.time()
for _ in range(num_docs):
    _ = encode_document(test_doc_text, doc_encoder, tokenizer, device)
elapsed = time.time() - start_time

print(f"총 시간: {elapsed:.4f}s")
print(f"평균 시간: {elapsed/num_docs*1000:.2f}ms per document")
print(f"처리량: {num_docs/elapsed:.2f} documents/sec")

print("\n" + "="*60)
print("💡 요약")
print("="*60)
print("✓ 쿼리 인코딩: 매우 빠름 (IDF lookup만 사용)")
print("✓ 문서 인코딩: 느림 (BERT 모델 사용)")
print("✓ 쿼리는 실시간 처리 가능, 문서는 사전 인덱싱 필요")

## 9. 커스텀 테스트

In [ ]:
# 사용자 정의 쿼리와 문서로 테스트
custom_query = "인공지능 LLM 모델 최적화 방법"  # 여기에 테스트할 쿼리 입력

custom_documents = [
    "LLM 모델 최적화를 위해서는 양자화, 프루닝, 지식 증류 등의 기법을 사용할 수 있습니다.",
    "한국어 음식 중에서 김치찌개는 가장 인기 있는 메뉴 중 하나입니다.",
    "대형 언어 모델의 추론 속도를 높이기 위해 KV 캐시와 배치 처리를 활용합니다.",
]  # 여기에 테스트할 문서 입력

print("="*60)
print("🧪 커스텀 검색 테스트")
print("="*60)
print(f"\n쿼리: {custom_query}")

# 쿼리 인코딩
query_vec = encode_query_inference_free(custom_query, tokenizer, idf_dict)

# 문서 인코딩 및 유사도 계산
results = []
for i, doc in enumerate(custom_documents):
    doc_vec = encode_document(doc, doc_encoder, tokenizer, device)
    similarity = compute_similarity(query_vec, doc_vec)
    results.append((i, similarity, doc))

# 유사도 순으로 정렬
results.sort(key=lambda x: x[1], reverse=True)

print("\n검색 결과:")
for rank, (doc_idx, score, doc_text) in enumerate(results, 1):
    print(f"\n[{rank}위] 유사도: {score:.4f}")
    print(f"     문서: {doc_text}")

## 10. 희소성(Sparsity) 분석

In [ ]:
print("="*60)
print("📊 희소성 분석")
print("="*60)

# 문서 벡터 희소성
doc_sparsities = []
for vec in doc_vectors:
    non_zero = np.count_nonzero(vec)
    sparsity = (1 - non_zero / len(vec)) * 100
    doc_sparsities.append(sparsity)

print(f"\n문서 벡터 희소성:")
print(f"  평균: {np.mean(doc_sparsities):.2f}%")
print(f"  최소: {np.min(doc_sparsities):.2f}%")
print(f"  최대: {np.max(doc_sparsities):.2f}%")
print(f"  표준편차: {np.std(doc_sparsities):.2f}%")

# 쿼리 벡터 희소성
query_sparsities = []
for vec in query_vectors:
    non_zero = np.count_nonzero(vec)
    sparsity = (1 - non_zero / len(vec)) * 100
    query_sparsities.append(sparsity)

print(f"\n쿼리 벡터 희소성:")
print(f"  평균: {np.mean(query_sparsities):.2f}%")
print(f"  최소: {np.min(query_sparsities):.2f}%")
print(f"  최대: {np.max(query_sparsities):.2f}%")
print(f"  표준편차: {np.std(query_sparsities):.2f}%")

print(f"\n💡 희소성이 높을수록 (100%에 가까울수록):")
print(f"   - 메모리 효율적")
print(f"   - 검색 속도 빠름")
print(f"   - 저장 공간 적음")

## 11. 결과 요약

In [ ]:
print("="*60)
print("✅ OpenSearch Korean Neural Sparse Model - Inference Test 완료")
print("="*60)

print("\n📊 테스트 요약:")
print(f"  • 문서 인코딩 평균 시간: {np.mean(encoding_times)*1000:.2f}ms")
print(f"  • 쿼리 인코딩 평균 시간: {np.mean(query_encoding_times)*1000:.4f}ms")
print(f"  • 속도 차이: {np.mean(encoding_times)/np.mean(query_encoding_times):.1f}배")
print(f"  • 문서 벡터 평균 희소성: {np.mean(doc_sparsities):.2f}%")
print(f"  • 쿼리 벡터 평균 희소성: {np.mean(query_sparsities):.2f}%")

print("\n🎯 핵심 특징:")
print("  ✓ Inference-Free 쿼리 인코딩 (매우 빠름)")
print("  ✓ 높은 희소성 (메모리 효율적)")
print("  ✓ 의미 기반 검색 가능")
print("  ✓ 한국어 최적화")

print("\n💡 OpenSearch 통합:")
print("  1. 문서는 사전에 인코딩하여 인덱싱")
print("  2. 쿼리는 실시간으로 IDF lookup")
print("  3. rank_features 필드로 저장")
print("  4. neural_sparse 쿼리로 검색")

print("\n" + "="*60)

## 12. 🚀 Transformers 스타일 래퍼 클래스 (간편한 사용)

Hugging Face Transformers처럼 간단하게 사용할 수 있는 통합 클래스를 제공합니다.

In [ ]:
class NeuralSparseSearchModel:
    """
    OpenSearch Neural Sparse 검색 모델 - Transformers 스타일 인터페이스

    사용 예제:
        model = NeuralSparseSearchModel.from_pretrained("./models/opensearch-korean-neural-sparse-v1")
        results = model.search(query="검색 쿼리", documents=["문서1", "문서2", ...])
    """

    def __init__(self, doc_encoder, tokenizer, idf_dict, device='cuda', max_length=128):
        self.doc_encoder = doc_encoder
        self.tokenizer = tokenizer
        self.idf_dict = idf_dict
        self.device = device
        self.max_length = max_length

        # Document 벡터 캐시 (선택적)
        self._doc_cache = {}

    @classmethod
    def from_pretrained(cls, model_path, device=None):
        """
        저장된 모델에서 로드 (Transformers 스타일)

        Args:
            model_path: 모델 디렉토리 경로
            device: 디바이스 ('cuda' 또는 'cpu')

        Returns:
            NeuralSparseSearchModel 인스턴스
        """
        import os

        if device is None:
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
        device = torch.device(device)

        # Config 로드
        with open(os.path.join(model_path, "config.json"), 'r', encoding='utf-8') as f:
            config = json.load(f)

        # Tokenizer 로드
        tokenizer = AutoTokenizer.from_pretrained(model_path)

        # IDF 딕셔너리 로드
        with open(os.path.join(model_path, "idf.json"), 'r', encoding='utf-8') as f:
            idf_dict = json.load(f)

        # Document Encoder 로드
        base_model = config['base_model']
        doc_encoder = OpenSearchDocEncoder(base_model)
        state_dict = torch.load(
            os.path.join(model_path, "pytorch_model.bin"),
            map_location=device
        )
        doc_encoder.load_state_dict(state_dict)
        doc_encoder = doc_encoder.to(device)
        doc_encoder.eval()

        max_length = config.get('max_seq_length', 128)

        return cls(doc_encoder, tokenizer, idf_dict, device, max_length)

    def encode_query(self, query):
        """
        쿼리를 sparse vector로 인코딩 (Inference-Free!)

        Args:
            query: 쿼리 텍스트 (str)

        Returns:
            sparse_vector: (vocab_size,) numpy array
        """
        # 토큰화
        tokens = self.tokenizer.encode(
            query,
            add_special_tokens=False,
            max_length=self.max_length,
            truncation=True
        )

        # IDF lookup
        sparse_vec = np.zeros(self.tokenizer.vocab_size)
        for token_id in tokens:
            token_str = self.tokenizer.decode([token_id])
            if token_str in self.idf_dict:
                sparse_vec[token_id] = self.idf_dict[token_str]

        return sparse_vec

    def encode_documents(self, documents, batch_size=8, show_progress=True):
        """
        문서들을 배치로 인코딩

        Args:
            documents: 문서 리스트
            batch_size: 배치 크기
            show_progress: 진행률 표시

        Returns:
            List of sparse vectors
        """
        from tqdm.auto import tqdm

        self.doc_encoder.eval()
        doc_vectors = []

        iterator = range(0, len(documents), batch_size)
        if show_progress:
            iterator = tqdm(iterator, desc="Encoding documents")

        for i in iterator:
            batch = documents[i:i+batch_size]

            # 배치 토큰화
            encoded = self.tokenizer(
                batch,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )

            input_ids = encoded['input_ids'].to(self.device)
            attention_mask = encoded['attention_mask'].to(self.device)

            # Inference
            with torch.no_grad():
                sparse_vecs = self.doc_encoder(input_ids, attention_mask)

            doc_vectors.extend(sparse_vecs.cpu().numpy())

        return doc_vectors

    def search(self, query, documents, top_k=5, return_scores=True, batch_size=8):
        """
        검색 수행

        Args:
            query: 검색 쿼리 (str)
            documents: 문서 리스트 또는 문서 벡터 리스트
            top_k: 반환할 상위 결과 수
            return_scores: 점수 포함 여부
            batch_size: 문서 인코딩 배치 크기

        Returns:
            DataFrame with columns: rank, document, score (if return_scores=True)
        """
        # 쿼리 인코딩
        query_vec = self.encode_query(query)

        # 문서 인코딩 (문자열인 경우)
        if len(documents) > 0 and isinstance(documents[0], str):
            doc_vectors = self.encode_documents(documents, batch_size=batch_size)
        else:
            doc_vectors = documents

        # 유사도 계산
        similarities = []
        for i, doc_vec in enumerate(doc_vectors):
            sim = np.dot(query_vec, doc_vec)
            similarities.append((i, sim))

        # 유사도 순으로 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 상위 k개 결과
        top_results = similarities[:top_k]

        # DataFrame 생성
        results = []
        for rank, (doc_idx, score) in enumerate(top_results, 1):
            if isinstance(documents[0], str):
                result = {
                    'rank': rank,
                    'document_id': doc_idx,
                    'document': documents[doc_idx],
                }
            else:
                result = {
                    'rank': rank,
                    'document_id': doc_idx,
                }

            if return_scores:
                result['score'] = score

            results.append(result)

        return pd.DataFrame(results)

    def get_top_tokens(self, sparse_vec, top_k=10):
        """
        Sparse vector에서 상위 토큰 추출

        Args:
            sparse_vec: (vocab_size,) numpy array
            top_k: 상위 k개

        Returns:
            DataFrame with columns: rank, token, score
        """
        top_indices = np.argsort(sparse_vec)[-top_k:][::-1]
        top_values = sparse_vec[top_indices]

        results = []
        for rank, (idx, val) in enumerate(zip(top_indices, top_values), 1):
            if val > 0:
                token = self.tokenizer.decode([idx])
                results.append({
                    'rank': rank,
                    'token': token,
                    'score': val
                })

        return pd.DataFrame(results)

    def compute_sparsity(self, sparse_vec):
        """
        희소성 계산

        Args:
            sparse_vec: (vocab_size,) numpy array

        Returns:
            sparsity percentage (float)
        """
        non_zero = np.count_nonzero(sparse_vec)
        sparsity = (1 - non_zero / len(sparse_vec)) * 100
        return sparsity


print("✓ NeuralSparseSearchModel 클래스 정의 완료")

## 13. 래퍼 클래스 사용 예제

In [ ]:
# 모델 로드 (Transformers 스타일!)
model = NeuralSparseSearchModel.from_pretrained(
    MODEL_DIR,
    device=device
)

print("✅ 모델 로드 완료!")
print(f"   디바이스: {model.device}")
print(f"   Vocab size: {model.tokenizer.vocab_size:,}")
print(f"   Max length: {model.max_length}")

### 13.1. 간단한 검색 예제

In [ ]:
# 검색할 문서 준비
search_documents = [
    "OpenSearch는 강력한 검색 및 분석 엔진입니다. Neural sparse 검색을 지원합니다.",
    "한국어 자연어 처리 기술이 급속도로 발전하고 있습니다.",
    "딥러닝 모델 학습에는 GPU가 필수적입니다.",
    "ChatGPT는 대화형 AI 모델입니다.",
    "벡터 검색은 의미 기반 검색을 가능하게 합니다.",
    "Python은 데이터 과학 분야에서 가장 인기 있는 언어입니다.",
    "Transformer 아키텍처는 NLP의 혁명을 일으켰습니다.",
    "검색 엔진 최적화는 웹사이트 트래픽 증대에 중요합니다.",
]

# 검색 수행
query = "OpenSearch 검색 엔진"
results = model.search(query, search_documents, top_k=3)

print(f"\n쿼리: '{query}'\n")
print(results.to_string(index=False))

### 13.2. 다중 쿼리 검색

In [ ]:
# 여러 쿼리로 검색
queries = [
    "딥러닝 GPU",
    "자연어 처리",
    "ChatGPT AI",
]

print("="*80)
print("🔍 다중 쿼리 검색 결과")
print("="*80)

for query in queries:
    print(f"\n쿼리: '{query}'")
    print("-" * 80)

    results = model.search(query, search_documents, top_k=3)
    print(results.to_string(index=False))
    print()

### 13.3. 쿼리/문서 분석

In [ ]:
# 쿼리 분석
query = "OpenSearch 벡터 검색 최적화"
query_vec = model.encode_query(query)

print(f"쿼리: '{query}'\n")
print(f"희소성: {model.compute_sparsity(query_vec):.2f}%")
print(f"\n상위 토큰:")
print(model.get_top_tokens(query_vec, top_k=10).to_string(index=False))

### 13.4. 배치 문서 인코딩

In [ ]:
# 대량 문서 인코딩
large_document_set = [
    f"문서 {i}: 이것은 테스트 문서입니다. Neural sparse 검색을 테스트합니다."
    for i in range(20)
]

print("📄 배치 문서 인코딩 중...\n")

# 배치로 인코딩 (progress bar 표시)
doc_vectors = model.encode_documents(large_document_set, batch_size=8)

print(f"\n✓ {len(doc_vectors)}개 문서 인코딩 완료!")
print(f"  평균 희소성: {np.mean([model.compute_sparsity(v) for v in doc_vectors]):.2f}%")

### 13.5. 사전 인코딩된 문서로 검색

In [ ]:
# 문서를 미리 인코딩해두고 반복적으로 검색
# (실제 시스템에서는 문서를 한 번만 인코딩하고 저장)

print("1️⃣ 문서 사전 인코딩...\n")
cached_doc_vectors = model.encode_documents(search_documents, batch_size=4)

print("\n2️⃣ 사전 인코딩된 문서로 빠른 검색...\n")

# 여러 쿼리로 빠르게 검색
test_queries = [
    "검색 엔진",
    "AI 모델",
    "딥러닝",
]

for query in test_queries:
    # 벡터만 전달하면 인코딩 생략 (매우 빠름!)
    results = model.search(query, cached_doc_vectors, top_k=2)

    print(f"쿼리: '{query}'")
    print("-" * 60)

    for _, row in results.iterrows():
        doc_idx = row['document_id']
        score = row['score']
        print(f"  [{row['rank']}위] 점수: {score:.4f}")
        print(f"       문서: {search_documents[doc_idx][:60]}...")
    print()

## 14. 성능 비교: 기존 vs 래퍼 클래스

In [ ]:
print("="*80)
print("⚡ 성능 비교: 기존 함수 vs 래퍼 클래스")
print("="*80)

test_docs = [f"테스트 문서 {i}" for i in range(50)]
test_query = "테스트"

# 1. 기존 방식
print("\n1️⃣ 기존 방식 (개별 함수 호출)")
start = time.time()
for doc in test_docs:
    _ = encode_document(doc, doc_encoder, tokenizer, device)
elapsed_old = time.time() - start
print(f"   시간: {elapsed_old:.4f}s")

# 2. 래퍼 클래스 (배치 처리)
print("\n2️⃣ 래퍼 클래스 (배치 처리)")
start = time.time()
_ = model.encode_documents(test_docs, batch_size=8, show_progress=False)
elapsed_new = time.time() - start
print(f"   시간: {elapsed_new:.4f}s")

print(f"\n💡 속도 향상: {elapsed_old/elapsed_new:.2f}배 빠름!")

## 15. 실전 예제: 한국어 뉴스 검색

In [ ]:
# 한국어 뉴스 예제
news_articles = [
    "삼성전자가 차세대 반도체 기술 개발에 성공했다고 발표했습니다.",
    "인공지능 기술이 의료 분야에서 혁신을 일으키고 있습니다.",
    "정부가 탄소중립 정책을 강화하기로 결정했습니다.",
    "자율주행 자동차 상용화가 2025년에 본격화될 전망입니다.",
    "K-pop이 전 세계적으로 큰 인기를 끌고 있습니다.",
    "바이오 기업들이 신약 개발에 박차를 가하고 있습니다.",
    "전기차 배터리 기술이 빠르게 발전하고 있습니다.",
    "메타버스가 차세대 인터넷 플랫폼으로 주목받고 있습니다.",
]

print("="*80)
print("📰 한국어 뉴스 검색 시스템")
print("="*80)

# 뉴스 쿼리
news_queries = [
    "반도체 기술",
    "AI 의료",
    "전기차 배터리",
]

for query in news_queries:
    print(f"\n🔍 검색어: '{query}'")
    print("-" * 80)

    results = model.search(query, news_articles, top_k=3)

    for _, row in results.iterrows():
        print(f"\n  [{row['rank']}위] 점수: {row['score']:.4f}")
        print(f"       {row['document']}")

## 16. 요약 및 사용 가이드

### ✅ NeuralSparseSearchModel 주요 기능

1. **`from_pretrained(model_path)`** - 모델 로드
2. **`encode_query(query)`** - 쿼리 인코딩 (Inference-Free, 매우 빠름)
3. **`encode_documents(documents, batch_size)`** - 문서 배치 인코딩
4. **`search(query, documents, top_k)`** - 검색 수행
5. **`get_top_tokens(sparse_vec, top_k)`** - 상위 토큰 분석
6. **`compute_sparsity(sparse_vec)`** - 희소성 계산

### 💡 사용 패턴

#### 패턴 1: 즉시 검색 (문서 인코딩 즉시 수행)
```python
model = NeuralSparseSearchModel.from_pretrained("./model")
results = model.search(query="검색어", documents=["문서1", "문서2"])
```

#### 패턴 2: 사전 인코딩 (대량 문서, 반복 검색)
```python
# 1회만 인코딩
doc_vectors = model.encode_documents(documents, batch_size=16)

# 빠른 검색 (여러 번)
results1 = model.search(query="쿼리1", documents=doc_vectors)
results2 = model.search(query="쿼리2", documents=doc_vectors)
```

### 🎯 실전 활용

- **OpenSearch 통합**: 문서를 미리 인코딩하여 rank_features 필드에 저장
- **실시간 검색**: 쿼리만 즉시 인코딩 (Inference-Free)
- **배치 처리**: 대량 문서는 배치로 효율적으로 인코딩
- **분석**: 상위 토큰 분석으로 검색 품질 개선